In [1]:
!pip install -U transformers accelerate bitsandbytes

     |████████████████████████████████| 10.4 MB 5.7 MB/s eta 0:00:01
     |████████████████████████████████| 354 kB 7.0 MB/s eta 0:00:01
     |████████████████████████████████| 105.0 MB 12.8 MB/s eta 0:00:01
     |████████████████████████████████| 2.7 MB 14.6 MB/s eta 0:00:01
     |████████████████████████████████| 481 kB 31.5 MB/s eta 0:00:01
     |████████████████████████████████| 284 kB 6.6 MB/s eta 0:00:01
     |████████████████████████████████| 418 kB 10.5 MB/s eta 0:00:01
You should consider upgrading via the '/Users/ron/Documents/github/myenv/bin/python3 -m pip install --upgrade pip' command.


In [12]:
!pip install -U sentencepiece

     |████████████████████████████████| 1.2 MB 1.9 MB/s eta 0:00:01
You should consider upgrading via the '/Users/ron/Documents/github/myenv/bin/python3 -m pip install --upgrade pip' command.


In [5]:
!pip install -U requests

You should consider upgrading via the '/Users/ron/Documents/github/myenv/bin/python3 -m pip install --upgrade pip' command.


In [1]:
!df -h

Filesystem                                     Size    Used   Avail Capacity iused ifree %iused  Mounted on
/dev/disk3s1s1                                926Gi    10Gi   830Gi     2%    425k  4.3G    0%   /
devfs                                         205Ki   205Ki     0Bi   100%     708     0  100%   /dev
/dev/disk3s6                                  926Gi   7.0Gi   830Gi     1%       7  8.7G    0%   /System/Volumes/VM
/dev/disk3s2                                  926Gi   6.6Gi   830Gi     1%    1.4k  8.7G    0%   /System/Volumes/Preboot
/dev/disk3s4                                  926Gi   5.3Mi   830Gi     1%      58  8.7G    0%   /System/Volumes/Update
/dev/disk1s2                                  500Mi   6.0Mi   481Mi     2%       1  4.9M    0%   /System/Volumes/xarts
/dev/disk1s1                                  500Mi   5.4Mi   481Mi     2%      29  4.9M    0%   /System/Volumes/iSCPreboot
/dev/disk1s3                                  500Mi   2.4Mi   481Mi     1%      91  4.9M   

In [2]:
import os
os.environ['HF_HOME'] = '/System/Volumes/Data/Users/ron/Documents/llm_models/'

In [3]:
#!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

In [4]:
import requests 
import minsearch

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

/Users/ron/Documents/github/myenv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [8]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [9]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [10]:
# pip install accelerate
from transformers import T5Tokenizer, T5ForConditionalGeneration

In [11]:
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-xl")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-xl", device_map="auto")

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.45G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Some parameters are on the meta device because they were offloaded to the disk.


In [12]:
input_text = "translate English to German: How old are you?"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("mps")

outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0]))

<pad> Wie alt sind Sie?</s>


In [13]:
input_ids

tensor([[13959,  1566,    12,  2968,    10,   571,   625,    33,    25,    58,
             1]], device='mps:0')

In [14]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

def llm(prompt):
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("mps")
    outputs = model.generate(input_ids )
    result = tokenizer.decode(outputs[0])
    return result

In [15]:
rag("I just discovered the course. Can I still join it?")

"<pad>Yes, even if you don't register, you're still eligible to submit the homework"

In [16]:
def llm(prompt, generate_params=None):
    if generate_params is None:
        generate_params = {}

    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("mps")
    outputs = model.generate(
        input_ids,
        max_length=generate_params.get("max_length", 100),
        num_beams=generate_params.get("num_beams", 5),
        do_sample=generate_params.get("do_sample", False),
        temperature=generate_params.get("temperature", 1.0),
        top_k=generate_params.get("top_k", 50),
        top_p=generate_params.get("top_p", 0.95),
    )
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return result

In [17]:
rag("I just discovered the course. Can I still join it?")

/Users/ron/Documents/github/myenv/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


"Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute."